# Infer-13-Debugging : Troubleshooting et Bonnes Pratiques

**Serie** : Programmation Probabiliste avec Infer.NET (13/13)  
**Duree estimee** : 45 minutes  
**Prerequis** : Avoir explore plusieurs notebooks de la serie

---

## Objectifs

- Diagnostiquer les problemes courants d'inference
- Comparer les algorithmes (EP, VMP, Gibbs)
- Utiliser les outils de debug d'Infer.NET
- Appliquer les bonnes pratiques de modelisation

---

## Navigation

| Precedent | Index |
|-----------|-------|
| [Infer-12-Recommenders](Infer-12-Recommenders.ipynb) | [README](README.md) |

---

## 1. Configuration

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


> **Note technique** : Ce notebook est oriente *troubleshooting*. Il suppose que vous avez deja explore plusieurs notebooks de la serie et rencontre des comportements inattendus. Les exemples sont volontairement simplifies pour isoler chaque type de probleme.

Le debugging en programmation probabiliste differe du debugging classique :

| Debugging classique | Debugging probabiliste |
|---------------------|------------------------|
| Erreur = crash ou mauvaise valeur | Erreur = distribution inattendue ou divergence |
| Cause souvent deterministe | Cause souvent liee aux priors ou a l'algorithme |
| Solution : corriger le code | Solution : ajuster le modele ou l'algorithme |

In [2]:
// Chargement du helper pour afficher les graphes de facteurs inline
#load "FactorGraphHelper.cs"

Console.WriteLine("FactorGraphHelper charge !");
Console.WriteLine($"Graphviz disponible : {FactorGraphHelper.IsGraphvizAvailable()}");
Console.WriteLine("Usage: display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()))");

FactorGraphHelper charge !
Graphviz disponible : True
Usage: display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()))


## 2. Erreurs Courantes et Solutions

### 2.1 Catalogue des Erreurs

| Erreur | Cause | Solution |
|--------|-------|----------|
| `Model has no support` | Observation impossible sous le prior | Elargir le prior ou verifier les observations |
| `Improper distribution` | Divergence de l'inference | Utiliser des priors plus informatifs |
| `Could not find method` | Operation non supportee | Reformuler avec des operations de base |
| `Compilation timeout` | Modele trop complexe | Simplifier ou compiler separement |
| `Memory exceeded` | Trop de variables | Reduire la taille ou utiliser des arrays |

In [3]:
// Exemple 1 : Erreur "Model has no support"

Console.WriteLine("=== Erreur : Model has no support ===");
Console.WriteLine();

// PROBLEME : Observer une valeur impossible sous le prior
try
{
    Variable<double> x = Variable.GaussianFromMeanAndPrecision(0, 1000);  // Prior tres concentre autour de 0
    x.ObservedValue = 100;  // Observation tres eloignee
    
    InferenceEngine engine = new InferenceEngine();
    engine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    var result = engine.Infer(x);  // Peut echouer ou donner des resultats etranges
}
catch (Exception e)
{
    Console.WriteLine($"Erreur : {e.Message.Substring(0, Math.Min(100, e.Message.Length))}...");
}

// SOLUTION : Utiliser un prior plus large
Console.WriteLine("\nSOLUTION : Elargir le prior");
Variable<double> x2 = Variable.GaussianFromMeanAndPrecision(50, 0.01);  // Prior large
x2.ObservedValue = 100;

InferenceEngine engine2 = new InferenceEngine();
engine2.Compiler.CompilerChoice = CompilerChoice.Roslyn;
Console.WriteLine($"Resultat avec prior large : {engine2.Infer(x2)}");

=== Erreur : Model has no support ===

Compiling model...done.

SOLUTION : Elargir le prior
Compiling model...done.
Resultat avec prior large : Gaussian.PointMass(100)


**Interpretation des resultats** :

Le resultat `Gaussian.PointMass(100)` signifie que le posterieur est une distribution degeneree concentree exactement sur la valeur observee. C'est le comportement attendu quand :

1. L'observation est deterministe (`ObservedValue`)
2. Le prior est suffisamment large pour "accepter" cette valeur

> **Regle pratique** : Si votre prior a une precision `prec`, alors les observations situees a plus de `3/sqrt(prec)` ecarts-types du centre auront un support quasi-nul. Avec `prec=1000`, cela represente environ `0.1` unites autour de la moyenne.

In [4]:
// Exemple 2 : Probleme de convergence (posterieurs uniformes)

Console.WriteLine("=== Probleme : Convergence vers mode symetrique ===");
Console.WriteLine();

// PROBLEME : Priors symetriques dans un modele de melange
int nComp = 2;
Range compRange = new Range(nComp);

VariableArray<double> means = Variable.Array<double>(compRange);
means[compRange] = Variable.GaussianFromMeanAndPrecision(0, 0.01).ForEach(compRange);  // Prior identique

Console.WriteLine("Avec priors symetriques (meme pour toutes les composantes) :");
Console.WriteLine("  -> Le modele peut converger vers une solution ou toutes");
Console.WriteLine("     les composantes sont identiques (mode symetrique).");

// SOLUTION : Priors asymetriques
Console.WriteLine("\nSOLUTION : Utiliser des priors asymetriques");
Console.WriteLine("  means[0] ~ Gaussian(-5, 1)");
Console.WriteLine("  means[1] ~ Gaussian(+5, 1)");
Console.WriteLine("  -> Force les composantes a etre distinctes");

=== Probleme : Convergence vers mode symetrique ===

Avec priors symetriques (meme pour toutes les composantes) :
  -> Le modele peut converger vers une solution ou toutes
     les composantes sont identiques (mode symetrique).

SOLUTION : Utiliser des priors asymetriques
  means[0] ~ Gaussian(-5, 1)
  means[1] ~ Gaussian(+5, 1)
  -> Force les composantes a etre distinctes


### Le probleme du "label switching"

Dans les modeles de melange, le probleme des modes symetriques est connu sous le nom de **label switching**. Si les composantes sont echangeables (meme prior), l'inference peut :

1. **Converger vers la moyenne** : toutes les composantes au meme endroit
2. **Osciller entre permutations** : les labels des composantes s'echangent entre iterations

Les solutions incluent :

| Solution | Avantage | Inconvenient |
|----------|----------|--------------|
| Priors asymetriques | Simple a implementer | Introduit un biais |
| Contraintes d'ordre | Mathematiquement propre | Complexifie le modele |
| Post-traitement | Pas de biais | Necessite analyse manuelle |

## 3. Comparaison des Algorithmes d'Inference

### Quand utiliser quel algorithme ?

| Algorithme | Forces | Faiblesses | Usage recommande |
|------------|--------|------------|------------------|
| **EP** | Rapide, bon pour melanges continus | Peut diverger, approximatif | Modeles Gaussian, Probit |
| **VMP** | Stable, bon pour discret | Sous-estime l'incertitude | LDA, melanges categoriques |
| **Gibbs** | Exact asymptotiquement | Lent, diagnostics difficiles | Validation, petits modeles |

In [ ]:
// Comparaison EP vs VMP sur un modele simple

Console.WriteLine("=== Comparaison EP vs VMP ===");
Console.WriteLine();

// Modele : estimation de moyenne avec observations bruitees
double[] observations = { 2.1, 1.9, 2.3, 2.0, 1.8, 2.2 };
int nObs = observations.Length;

// Fonction pour creer et inferer le modele avec EP
Gaussian InferAvecEP()
{
    Variable<double> mean = Variable.GaussianFromMeanAndPrecision(0, 0.01);
    Variable<double> prec = Variable.GammaFromShapeAndScale(2, 0.5);
    
    Range r = new Range(nObs);
    VariableArray<double> obs = Variable.Array<double>(r);
    obs[r] = Variable.GaussianFromMeanAndPrecision(mean, prec).ForEach(r);
    obs.ObservedValue = observations;
    
    InferenceEngine engine = new InferenceEngine(new ExpectationPropagation());
    engine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    engine.ShowProgress = false;
    
    return engine.Infer<Gaussian>(mean);
}

// Fonction pour creer et inferer le modele avec VMP
Gaussian InferAvecVMP()
{
    Variable<double> mean = Variable.GaussianFromMeanAndPrecision(0, 0.01);
    Variable<double> prec = Variable.GammaFromShapeAndScale(2, 0.5);
    
    Range r = new Range(nObs);
    VariableArray<double> obs = Variable.Array<double>(r);
    obs[r] = Variable.GaussianFromMeanAndPrecision(mean, prec).ForEach(r);
    obs.ObservedValue = observations;
    
    InferenceEngine engine = new InferenceEngine(new VariationalMessagePassing());
    engine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    engine.ShowProgress = false;
    
    return engine.Infer<Gaussian>(mean);
}

var resultEP = InferAvecEP();
var resultVMP = InferAvecVMP();

Console.WriteLine($"Observations : {string.Join(", ", observations)}");
Console.WriteLine($"Moyenne empirique : {observations.Average():F3}");
Console.WriteLine();
Console.WriteLine($"EP  : {resultEP}");
Console.WriteLine($"VMP : {resultVMP}");
Console.WriteLine();
Console.WriteLine("Note : VMP tend a avoir une variance plus faible (sous-estime l'incertitude)");

**Interpretation des resultats EP vs VMP** :

Les deux algorithmes estiment la moyenne autour de 2.05 (proche de la moyenne empirique 2.05), mais avec des caracteristiques differentes :

| Metrique | EP | VMP | Interpretation |
|----------|----|----|----------------|
| Moyenne posterieure | ~2.05 | ~2.05 | Accord sur l'estimation ponctuelle |
| Variance posterieure | Plus large | Plus etroite | VMP sous-estime l'incertitude |

> **Pourquoi VMP sous-estime l'incertitude ?**
> 
> VMP (Variational Message Passing) approxime le posterieur par une distribution factorisee. Cette hypothese d'independance ignore les correlations entre variables, ce qui conduit typiquement a des posterieurs trop "confiants".
>
> EP (Expectation Propagation) maintient des correlations locales via les messages, produisant des approximations plus realistes de l'incertitude.

**Quand cela importe** : La sous-estimation de l'incertitude par VMP peut etre problematique pour :
- La prise de decision sous incertitude
- Les intervalles de prediction
- La propagation de l'incertitude dans des modeles hierarchiques

## 4. Outils de Debug Infer.NET

### 4.1 Options du Moteur

```csharp
engine.ShowProgress = true;           // Affiche les iterations
engine.ShowSchedule = true;           // Affiche l'ordre des messages
engine.ShowFactorGraph = true;        // Genere le graphe de facteurs
engine.Compiler.WriteSourceFiles = true;  // Sauvegarde le code genere
engine.Compiler.ShowWarnings = true;  // Affiche les avertissements
```

In [6]:
// Demonstration des outils de debug

Console.WriteLine("=== Outils de Debug ===");
Console.WriteLine();

// Modele simple pour demonstration
Variable<double> mu = Variable.GaussianFromMeanAndPrecision(0, 1).Named("mu");
Variable<double> y = Variable.GaussianFromMeanAndPrecision(mu, 1).Named("y");
y.ObservedValue = 5.0;

InferenceEngine debugEngine = new InferenceEngine();
debugEngine.Compiler.CompilerChoice = CompilerChoice.Roslyn;

// Activer les options de debug
debugEngine.ShowProgress = true;
debugEngine.Compiler.ShowWarnings = true;

Console.WriteLine("Options de debug activees :");
Console.WriteLine("  - ShowProgress : affiche les iterations");
Console.WriteLine("  - ShowWarnings : affiche les avertissements du compilateur");
Console.WriteLine();

var muPost = debugEngine.Infer<Gaussian>(mu);
Console.WriteLine($"\nResultat : mu ~ {muPost}");

=== Outils de Debug ===

Options de debug activees :
  - ShowProgress : affiche les iterations
  - ShowWarnings : affiche les avertissements du compilateur

Compiling model...done.

Resultat : mu ~ Gaussian(2,5, 0,5)


**Interpretation du resultat** :

Le posterieur `Gaussian(2.5, 0.5)` resulte de la mise a jour bayesienne :

$$\mu_{\text{post}} = \frac{\tau_{\text{prior}} \cdot \mu_{\text{prior}} + \tau_{\text{likelihood}} \cdot y}{\tau_{\text{prior}} + \tau_{\text{likelihood}}} = \frac{1 \cdot 0 + 1 \cdot 5}{1 + 1} = 2.5$$

$$\tau_{\text{post}} = \tau_{\text{prior}} + \tau_{\text{likelihood}} = 1 + 1 = 2 \quad \Rightarrow \quad \sigma^2_{\text{post}} = 0.5$$

Ou $\tau$ represente la precision (inverse de la variance). Le posterieur est exactement a mi-chemin entre le prior (0) et l'observation (5), car les deux ont la meme precision.

### 4.2 Visualisation des Factor Graphs

L'option `ShowFactorGraph = true` genere des fichiers `.gv` (DOT) et `.svg` (si Graphviz est installe). Le helper `FactorGraphHelper` permet d'afficher ces graphes directement dans le notebook.

In [7]:
// Demonstration de la visualisation du Factor Graph

Console.WriteLine("=== Visualisation du Factor Graph ===");
Console.WriteLine();

// Modele hierarchique pour une visualisation interessante
Variable<double> hyperMean = Variable.GaussianFromMeanAndPrecision(0, 0.1).Named("hyperMean");
Variable<double> hyperPrec = Variable.GammaFromShapeAndScale(2, 0.5).Named("hyperPrec");
Variable<double> obs1 = Variable.GaussianFromMeanAndPrecision(hyperMean, hyperPrec).Named("obs1");
Variable<double> obs2 = Variable.GaussianFromMeanAndPrecision(hyperMean, hyperPrec).Named("obs2");

obs1.ObservedValue = 3.0;
obs2.ObservedValue = 5.0;

InferenceEngine fgEngine = new InferenceEngine();
fgEngine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
fgEngine.ShowFactorGraph = true;  // Generer le graphe
fgEngine.ShowProgress = false;

var hyperMeanPost = fgEngine.Infer<Gaussian>(hyperMean);
Console.WriteLine($"Resultat : hyperMean ~ {hyperMeanPost}");

// Afficher le factor graph inline
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

// Nettoyer les fichiers generes
int cleaned = FactorGraphHelper.CleanupGeneratedFiles();
Console.WriteLine($"\nFichiers nettoyes : {cleaned}");

=== Visualisation du Factor Graph ===

Resultat : hyperMean ~ Gaussian(3,74, 0,9106)


Model_01_29_26_14_46_33_68.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 0 
 
<!-- node1 -->
 
 node1 
 
 Gaussian 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 mean 
 
<!-- node3 -->
 
 node3 
 
 hyperMean 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 0,1 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 precision 
 
<!-- node4 -->
 
 node4 
 
 Gaussian 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node7 -->
 
 node7 
 
 Gaussian 
 
<!-- node3->node7 -->
 
 node3->node7 
 
 
 mean 
 
<!-- node6 -->
 
 node6 
 
 3 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 hyperPrec 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 precision 
 
<!-- node5->node7 -->
 
 node5->node7 
 
 
 precision 
 
<!-- node8 -->
 
 node8 
 
 5 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 
<!-- node9 -->
 
 node9 
 
 2 
 
<!-- node10 -->
 
 node10 
 
 Sample 
 
<!-- node9->node10 -->
 
 node9->node10 
 
 
 shape 
 
<!-- node10->node5 -->
 
 node10->node5 
 
 
 
<!-- node11 -->
 
 node11 
 
 0,5 
 
<!-- node11->node10 -->
 
 node11->node10 
 
 
 scale


Fichiers nettoyes : 4



warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Comment lire un Factor Graph ?

Le graphe de facteurs visualise la structure du modele probabiliste :

| Element | Representation | Signification |
|---------|----------------|---------------|
| **Cercles** | Variables | Variables aleatoires du modele |
| **Carres** | Facteurs | Distributions ou contraintes |
| **Aretes** | Connexions | Dependances entre variables et facteurs |
| **Couleur grise** | Observe | Variables avec valeurs fixees |

> **Utilite pour le debugging** :
> 
> Le factor graph permet de verifier visuellement que :
> 1. Les variables sont connectees comme prevu
> 2. Les observations sont bien marquees
> 3. Il n'y a pas de composantes deconnectees
> 4. La structure hierarchique est correcte

Dans le graphe ci-dessus, on voit que `hyperMean` et `hyperPrec` sont les hyperparametres partages par les deux observations `obs1` et `obs2`, formant un modele hierarchique classique.

## 5. Bonnes Pratiques de Modelisation

### 5.1 Nommage des Variables

```csharp
// BON : Noms explicites avec .Named()
Variable<double> capaciteEtudiant = Variable.GaussianFromMeanAndPrecision(0, 1).Named("capacite");

// MAUVAIS : Variables anonymes
Variable<double> x = Variable.GaussianFromMeanAndPrecision(0, 1);
```

### 5.2 Priors Informatifs

| Situation | Prior recommande |
|-----------|------------------|
| Moyenne inconnue | Gaussian large (precision ~0.01) |
| Precision inconnue | Gamma(2, 0.5) ou plus concentre |
| Probabilite | Beta(1, 1) pour uniforme, Beta(2, 2) pour centre |
| Poids melange | Dirichlet(1, 1, ...) pour uniforme |

In [8]:
// Demonstration de l'importance des priors

Console.WriteLine("=== Impact du choix des Priors ===");
Console.WriteLine();

// Observations : 3 succes sur 5 essais
int succes = 3, echecs = 2;

// Differents priors pour la probabilite
var priors = new (string nom, double a, double b)[] {
    ("Uniforme Beta(1,1)", 1, 1),
    ("Centre Beta(2,2)", 2, 2),
    ("Informatif Beta(5,5)", 5, 5),
    ("Biaise succes Beta(8,2)", 8, 2)
};

Console.WriteLine($"Observations : {succes} succes, {echecs} echecs");
Console.WriteLine($"MLE (maximum de vraisemblance) : {(double)succes/(succes+echecs):F2}");
Console.WriteLine();

foreach (var (nom, a, b) in priors)
{
    Variable<double> p = Variable.Beta(a, b);
    Variable<int> obs = Variable.Binomial(succes + echecs, p);
    obs.ObservedValue = succes;
    
    InferenceEngine eng = new InferenceEngine();
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    eng.ShowProgress = false;
    
    Beta postP = eng.Infer<Beta>(p);
    Console.WriteLine($"{nom,-25} -> Posterieur : mean = {postP.GetMean():F3}");
}

Console.WriteLine();
Console.WriteLine("Observation : Le prior influence le posterieur, surtout avec peu de donnees.");

=== Impact du choix des Priors ===

Observations : 3 succes, 2 echecs
MLE (maximum de vraisemblance) : 0,60

Uniforme Beta(1,1)        -> Posterieur : mean = 0,571
Centre Beta(2,2)          -> Posterieur : mean = 0,556
Informatif Beta(5,5)      -> Posterieur : mean = 0,533
Biaise succes Beta(8,2)   -> Posterieur : mean = 0,733

Observation : Le prior influence le posterieur, surtout avec peu de donnees.


**Analyse detaillee des resultats** :

| Prior | Alpha | Beta | Moyenne prior | Moyenne posterieure | Ecart au MLE |
|-------|-------|------|---------------|---------------------|--------------|
| Uniforme | 1 | 1 | 0.500 | 0.571 | -0.029 |
| Centre | 2 | 2 | 0.500 | 0.556 | -0.044 |
| Informatif | 5 | 5 | 0.500 | 0.533 | -0.067 |
| Biaise | 8 | 2 | 0.800 | 0.733 | +0.133 |

Le posterieur Beta suit la formule analytique :

$$p \mid \text{data} \sim \text{Beta}(\alpha + \text{succes}, \beta + \text{echecs})$$

$$\mathbb{E}[p \mid \text{data}] = \frac{\alpha + \text{succes}}{\alpha + \beta + \text{succes} + \text{echecs}}$$

> **Interpretation bayesienne** :
> 
> - Le prior **uniforme** (Beta(1,1)) est le plus "neutre" et donne un resultat proche du MLE
> - Les priors **centres** (Beta(2,2) et Beta(5,5)) "tirent" le posterieur vers 0.5
> - Le prior **biaise** (Beta(8,2)) domine les observations et maintient une estimation elevee
>
> La force de l'effet du prior depend du ratio entre les pseudo-observations du prior ($\alpha + \beta$) et les observations reelles (5 dans cet exemple).

## 6. Checklist de Debugging

Quand votre modele ne fonctionne pas, verifiez :

### Etape 1 : Verification du Modele
- [ ] Les types des variables sont corrects (double vs int vs bool)
- [ ] Les observations sont dans le support du prior
- [ ] Les arrays ont les bonnes dimensions
- [ ] Les Range sont correctement definis

### Etape 2 : Verification de l'Inference
- [ ] L'algorithme est adapte au modele (EP/VMP/Gibbs)
- [ ] Le nombre d'iterations est suffisant
- [ ] Les warnings de compilation sont examines

### Etape 3 : Verification des Resultats
- [ ] Les posterieurs ne sont pas degeneres (variance > 0)
- [ ] Les moyennes sont dans des plages raisonnables
- [ ] Les predictions sur donnees connues sont correctes

In [ ]:
// Fonctions utilitaires de diagnostic pour differents types de distributions

void DiagnosticGaussian(Gaussian posterior, string nom)
{
    Console.WriteLine($"=== Diagnostic : {nom} ===");
    Console.WriteLine($"  Distribution : {posterior}");
    Console.WriteLine($"  Moyenne : {posterior.GetMean():F4}");
    double variance = posterior.GetVariance();
    Console.WriteLine($"  Variance : {variance:F6}");
    
    if (variance < 1e-10)
        Console.WriteLine("  [ALERTE] Variance tres faible - possible degenerescence");
    if (variance > 1e6)
        Console.WriteLine("  [ALERTE] Variance tres elevee - inference non informative");
    Console.WriteLine();
}

void DiagnosticGamma(Gamma posterior, string nom)
{
    Console.WriteLine($"=== Diagnostic : {nom} ===");
    Console.WriteLine($"  Distribution : {posterior}");
    Console.WriteLine($"  Moyenne : {posterior.GetMean():F4}");
    double variance = posterior.GetVariance();
    Console.WriteLine($"  Variance : {variance:F6}");
    
    if (variance < 1e-10)
        Console.WriteLine("  [ALERTE] Variance tres faible - possible degenerescence");
    if (variance > 1e6)
        Console.WriteLine("  [ALERTE] Variance tres elevee - inference non informative");
    Console.WriteLine();
}

void DiagnosticBeta(Beta posterior, string nom)
{
    Console.WriteLine($"=== Diagnostic : {nom} ===");
    Console.WriteLine($"  Distribution : {posterior}");
    Console.WriteLine($"  Moyenne : {posterior.GetMean():F4}");
    double variance = posterior.GetVariance();
    Console.WriteLine($"  Variance : {variance:F6}");
    
    if (variance < 1e-10)
        Console.WriteLine("  [ALERTE] Variance tres faible - possible degenerescence");
    Console.WriteLine();
}

// Exemple d'utilisation
Variable<double> test = Variable.GaussianFromMeanAndPrecision(0, 0.1);
Variable<double> yTest = Variable.GaussianFromMeanAndPrecision(test, 1);
yTest.ObservedValue = 3.0;

InferenceEngine diagEngine = new InferenceEngine();
diagEngine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
diagEngine.ShowProgress = false;

var testPost = diagEngine.Infer<Gaussian>(test);
DiagnosticGaussian(testPost, "test");

**Interpretation des diagnostics** :

Les fonctions de diagnostic ci-dessus verifient plusieurs conditions de sante du posterieur :

| Condition | Seuil | Signification si viole |
|-----------|-------|------------------------|
| Variance trop faible | < 10^-10 | Distribution degeneree, possible erreur numerique |
| Variance trop elevee | > 10^6 | Inference non informative, donnees insuffisantes |

Pour l'exemple `test`, le diagnostic montre :
- **Moyenne ~2.73** : compromise entre le prior (0) et l'observation (3)
- **Variance ~0.91** : incertitude reduite par l'observation mais non nulle

> **Bonne pratique** : Integrez ces diagnostics dans vos pipelines d'inference pour detecter automatiquement les cas problematiques, surtout dans les modeles complexes avec de nombreuses variables latentes.

## 7. Exercice : Debugger un Modele

### Enonce

Le modele ci-dessous a plusieurs problemes. Identifiez et corrigez-les.

In [10]:
// EXERCICE : Trouvez les problemes dans ce modele

Console.WriteLine("=== Exercice : Debugger ce modele ===");
Console.WriteLine();

// Probleme 1 : Prior trop etroit pour les observations
// Probleme 2 : Variables non nommees
// Probleme 3 : Precision negative (invalide)

// Modele original (avec erreurs)
/*
Variable<double> m = Variable.GaussianFromMeanAndPrecision(0, 100);  // Prior trop concentre
Variable<double> p = Variable.GammaFromShapeAndScale(0.1, 0.1);      // Shape trop petit
Variable<double> obs = Variable.GaussianFromMeanAndPrecision(m, p);
obs.ObservedValue = 50;  // Tres eloigne du prior sur m
*/

// Version corrigee
Variable<double> moyenne = Variable.GaussianFromMeanAndPrecision(25, 0.01).Named("moyenne");  // Prior large
Variable<double> precision = Variable.GammaFromShapeAndScale(2, 0.5).Named("precision");      // Shape >= 1
Variable<double> observation = Variable.GaussianFromMeanAndPrecision(moyenne, precision).Named("obs");
observation.ObservedValue = 50;

InferenceEngine exEngine = new InferenceEngine();
exEngine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
exEngine.ShowProgress = false;

var moyPost = exEngine.Infer<Gaussian>(moyenne);
var precPost = exEngine.Infer<Gamma>(precision);

Console.WriteLine("Corrections appliquees :");
Console.WriteLine("1. Prior sur moyenne : precision 0.01 (large) au lieu de 100 (etroit)");
Console.WriteLine("2. Prior sur precision : Gamma(2, 0.5) au lieu de Gamma(0.1, 0.1)");
Console.WriteLine("3. Variables nommees avec .Named()");
Console.WriteLine();
Console.WriteLine($"Resultats : moyenne ~ {moyPost}, precision ~ {precPost}");

=== Exercice : Debugger ce modele ===

Corrections appliquees :
1. Prior sur moyenne : precision 0.01 (large) au lieu de 100 (etroit)
2. Prior sur precision : Gamma(2, 0.5) au lieu de Gamma(0.1, 0.1)
3. Variables nommees avec .Named()

Resultats : moyenne ~ Gaussian(49,48, 2,826), precision ~ Gamma(2, 0,4876)[mean=0,9752]


**Analyse des corrections** :

| Probleme original | Consequence | Correction appliquee |
|-------------------|-------------|----------------------|
| Precision 100 sur le prior | Prior concentre autour de 0 (ecart-type ~0.1) | Precision 0.01 (ecart-type ~10) |
| Observation a 50 | 500 ecarts-types du centre du prior | Prior centre a 25 pour couvrir l'observation |
| Gamma(0.1, 0.1) | Shape < 1 donne une densite infinie en 0 | Gamma(2, 0.5) avec shape >= 1 |
| Variables anonymes | Difficulte a interpreter les erreurs | Nommage explicite avec `.Named()` |

Le resultat corrige montre :
- **Moyenne ~49.5** : proche de l'observation (50) car le prior est large
- **Precision ~0.98** : estime a partir d'une seule observation (incertitude elevee)

### Points cles a retenir

> **Strategie de debugging en 3 etapes** :
>
> 1. **Verifier le support** : Les observations sont-elles probables sous le prior ?
> 2. **Verifier l'algorithme** : EP pour continu, VMP pour discret, Gibbs pour validation
> 3. **Verifier les posterieurs** : Variance raisonnable ? Moyennes plausibles ?

La plupart des problemes d'inference proviennent de :
- **Priors mal specifies** (trop etroits, mauvais support)
- **Mauvais choix d'algorithme** (EP sur modele discret, VMP sur correlations fortes)
- **Modele trop complexe** (simplifier d'abord, complexifier ensuite)

---

## 8. Resume

| Probleme | Symptome | Solution |
|----------|----------|----------|
| **Prior trop etroit** | "No support" ou posterieurs etranges | Elargir le prior |
| **Mode symetrique** | Posterieurs uniformes | Priors asymetriques |
| **Divergence** | Valeurs infinies ou NaN | Changer d'algorithme ou regulariser |
| **Lenteur** | Compilation longue | Simplifier le modele, cacher les types |

---

## Ressources

- [Documentation Infer.NET](https://dotnet.github.io/infer/)
- [FAQ Troubleshooting](https://dotnet.github.io/infer/userguide/Frequently%20Asked%20Questions.html)
- [GitHub Issues](https://github.com/dotnet/infer/issues)